In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

from farmwise_client import AgentClient

client = AgentClient(base_url="http://127.0.0.1:8082")


In [15]:

await client.ainvoke("What maize seed should I grow?", user_id="+254748256531")
# await client.ainvoke("I'm in Nakuru", user_id="+254748256530")

ChatMessage(type='assistant', content='Could you please provide your location in Kenya so I can suggest the best maize seed variety for your area?', actions=[<Action.request_location: 'request_location'>])

In [3]:
from farmwise.tools import soil_property

await soil_property(**{
    "ctx": None,
    "latitude": -1.0849843025208,
    "longitude": 35.878883361816,
    "property_": "ph"
})

{'property': {'ph': [{'value': {'unit': None, 'type': 'float', 'value': 7.2}, 'depth': {'value': '0-20', 'unit': 'cm'}, 'uncertainty': [{'confidence_interval': '50%', 'lower_bound': 7.1, 'upper_bound': 7.3}, {'confidence_interval': '68%', 'lower_bound': 7.1, 'upper_bound': 7.3}, {'confidence_interval': '90%', 'lower_bound': 7.0, 'upper_bound': 7.4}]}]}}


TypeError: list indices must be integers or slices, not str

In [7]:
d = {'property': {'ph':
    [
        {'value': {'unit': None, 'type': 'float', 'value': 7.2},
         'depth': {'value': '0-20', 'unit': 'cm'},
         'uncertainty': [{'confidence_interval': '50%', 'lower_bound': 7.1, 'upper_bound': 7.3},
                         {'confidence_interval': '68%', 'lower_bound': 7.1, 'upper_bound': 7.3},
                         {'confidence_interval': '90%', 'lower_bound': 7.0, 'upper_bound': 7.4}
                         ]}
    ]}}
d["property"]['ph'][0]["value"]["value"]

7.2

In [5]:
from farmwise_schema.schema import SectionList, Section, SectionRow

SectionList(button_title="Select disease", sections=[
    Section(title="Diseases", rows=[

        SectionRow(title="Maize lethal necrosis", callback_data="Maize lethal necrosis (MLN)"),
        SectionRow(title="Grey leaf spot", callback_data="Grey leaf spot (GLS)"),
        SectionRow(title="Northern leaf blight",  callback_data="Northern (Turcicum) leaf blight (NCLB)"),
        SectionRow(title="Common & southern rusts", callback_data="Common & southern rusts"),
        SectionRow(title="Stalk and ear rots", callback_data="Stalk and ear rots"),
        SectionRow(title="Maize streak virus", callback_data="Maize streak virus (MSV)"),
        SectionRow(title="Downy mildew", callback_data="Downy mildew"),
        SectionRow(title="Tar spot", callback_data="Tar spot"),

    ])
])

SectionList(button_title="Select pests", sections=[
    Section(title="Pests", rows=[

        SectionRow(title="Fall armyworm", callback_data="Fall armyworm"),
        SectionRow(title="Stem borers", callback_data="Stem borers"),
        SectionRow(title="Cutworms & earworms", callback_data="Cutworms & earworms"),
    ])
])


SectionList(button_title='Select pests', sections=[Section(title='Pests', rows=[SectionRow(title='Fall armyworm', callback_data='Fall armyworm'), SectionRow(title='Stem borers', callback_data='Stem borers'), SectionRow(title='Cutworms & earworms', callback_data='Cutworms & earworms')])])